In [1]:
import pickle
import numpy as np
import pandas as pd
import cupy as cp
import time
from tqdm import tqdm

## synthetic data(pkl파일)와 data(csv파일) 두 데이터셋에서 각 행을 비교해서 동일한 행이 몇 개인지 확인

## Dataframe

In [2]:

dtypes = {'age':'int','workclass':'category','education':'category','marital_status':'category','occupation':'category','relationship':'category','race':'category',
          'gender':'category','capital_gain':'int','capital_loss':'int','hours_per_week':'int','native_country':'category','income':'category'}
columns = list(dtypes.keys())
data = pd.read_csv('adultSalary.csv', names=columns).astype(dtypes)

data

,age,workclass,education,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,Bachelors,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
48838,64,?,HS-grad,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K
48839,38,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
48840,44,Private,Bachelors,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


In [3]:
n = 20000
n = len(data)
data = data[:n]

### df_synthpop

In [ ]:
# pkl 파일 경로를 지정합니다.
df_synthpop_pkl = '/home/casey/workspace/BigDataSystem/df_synthpop.pickle'
# 파일을 읽기 모드(rb)로 열어서 데이터를 로드합니다.
with open(df_synthpop_pkl, 'rb') as f:
    synth_data = pickle.load(f)
    
# 데이터를 확인합니다.
synth_data


FileNotFoundError: [Errno 2] No such file or directory: '/home/q/casey/BigDataSystem/df_synthpop.pickle'

In [6]:
import pandas as pd
import time
from tqdm import tqdm

# 데이터를 원본 그대로 유지하면서 인덱스를 재설정하지 않도록 하여 비교
data_list = []
synth_data_list = []

# 원본 데이터와 합성 데이터의 인덱스를 맞추는 과정
data = data.reset_index(drop=True)
synth_data = synth_data.reset_index(drop=True)

# 얘 땜에 index 변경됨
# synth_data = synth_data[:n-int(n*0.1)]
# synth_data = pd.concat([data[:int(n*0.1)], synth_data])

# 행 데이터를 리스트로 변환
stime = time.time()
for i in range(len(synth_data)):
    data_list.append(data.iloc[i, :].values.flatten().tolist())
    synth_data_list.append(synth_data.iloc[i, :].values.flatten().tolist())

cnt = 0
for row1 in tqdm(synth_data_list):
    for row2 in data_list:
        if row1 == row2:
            cnt += 1
            break

print("List 방식 시간:", time.time() - stime)
print("중복률:", cnt / len(synth_data))

# 인덱스 확인
print(data.index.equals(synth_data.index))  # True로 나와야 함


100%|██████████| 48842/48842 [03:14<00:00, 250.71it/s]

List 방식 시간: 200.10972094535828
중복률: 0.4122886040702674
True


### list로 바꿔서 해보기
- 이렇게 하면 원하는 값이 있는지 확인할 수 있음
- DataFrame과 List 속도 차이 : List, array로 바꿔서 사용하면 더 빠름

In [33]:
print(data.index.equals(synth_data.index))

True


### 해시 (set) + 튜플 방식

In [ ]:
stime = time.time()

# 1) 원본 행을 튜플로 바꿔 set에 저장
orig_tuples = set(map(tuple, data.values))
print(data.values)
print("========")
print(synth_data.values)
# 2) 합성 행을 튜플로 바꿔서 in 연산
cnt = 0
for row in synth_data.values:
    if tuple(row) in orig_tuples:
        cnt += 1

print("시간:", time.time() - stime) 
print("중복률:", cnt / len(synth_data)) 
print(cnt)


[[39 'State-gov' 'Bachelors' ... 40 'United-States' '<=50K']
 [50 'Self-emp-not-inc' 'Bachelors' ... 13 'United-States' '<=50K']
 [38 'Private' 'HS-grad' ... 40 'United-States' '<=50K']
 ...
 [38 'Private' 'Bachelors' ... 50 'United-States' '<=50K']
 [44 'Private' 'Bachelors' ... 40 'United-States' '<=50K']
 [35 'Self-emp-inc' 'Bachelors' ... 60 'United-States' '>50K']]
[[67 '?' 'HS-grad' ... 4 'United-States' '<=50K']
 [21 'Private' 'Some-college' ... 40 'United-States' '<=50K']
 [55 'Self-emp-not-inc' 'Bachelors' ... 30 'United-States' '<=50K']
 ...
 [71 'Self-emp-inc' 'Some-college' ... 24 'United-States' '<=50K']
 [25 'Private' 'HS-grad' ... 40 'United-States' '<=50K']
 [51 'Private' '10th' ... 30 'United-States' '<=50K']]
시간: 0.11937212944030762
중복률: 0.4122886040702674
20137


### Numba

In [8]:
import numba

# Numba JIT를 사용한 중복 체크 함수 (병렬 처리)
@numba.jit(nopython=True, parallel=True)
def check_duplicates(data, synth_data):
    n_s = len(synth_data)
    n_o = len(data)
    cnt = 0
    for i in numba.prange(n_s):  # 병렬화
        for j in range(n_o):
            if np.all(data[j] == synth_data[i]):
                cnt += 1
                break  # 중복을 찾으면 더 이상 비교하지 않음
    return cnt

# 모든 범주형 데이터를 원핫 인코딩
data_encoded = pd.get_dummies(data)
synth_encoded = pd.get_dummies(synth_data)

# 컬럼이 완전히 같도록 정렬 (중요!)
synth_encoded = synth_encoded.reindex(columns=data_encoded.columns, fill_value=0)
print(synth_encoded.index.equals(data_encoded.index))

# pandas DataFrame을 NumPy 배열로 변환할 때, dtype을 명시적으로 지정 (float32로 변환)
data_numpy = data_encoded.to_numpy(dtype=np.float32)  # float32로 변환
synth_data_numpy = synth_encoded.to_numpy(dtype=np.float32)  # float32로 변환


# 첫 번째 실행 시간 측정 (JIT 컴파일 포함)
stime = time.time()
duplicates_1 = check_duplicates(data_numpy, synth_data_numpy)
etime = time.time()
print(f"첫 번째 실행 시간: {etime - stime:.4f} 초")

# 두 번째 실행 시간 측정 (컴파일된 코드가 캐시됨)
stime = time.time()
duplicates_2 = check_duplicates(data_numpy, synth_data_numpy)
etime = time.time()
print(f"두 번째 실행 시간: {etime - stime:.4f} 초")

# 중복률 출력
print(f"중복률: {duplicates_1 / len(synth_data_numpy):.4f}")


True
첫 번째 실행 시간: 34.2307 초
두 번째 실행 시간: 33.8164 초
중복률: 0.4123


아니 두번째 실행에서 엄청 빨라질 줄 알았더니 왜 안됨?

### Numpy

In [9]:
stime = time.time()
a = data.to_numpy()
b = synth_data.to_numpy()

cnt = sum([np.any(np.all(row == a, axis=1)) for row in b])

print("Numpy 방식 시간:", time.time() - stime)
print("중복률:", cnt / len(b))
# Numpy 방식 시간: 231.23529720306396
# 중복률: 0.47049670365668894

Numpy 방식 시간: 238.78452682495117
중복률: 0.4122886040702674


### Cupy

In [73]:
import pandas as pd
import cupy as cp
import time

# 모든 범주형 데이터를 원핫 인코딩
data_encoded = pd.get_dummies(data)
synth_encoded = pd.get_dummies(synth_data)

# 컬럼이 완전히 같도록 정렬 (중요!)
synth_encoded = synth_encoded.reindex(columns=data_encoded.columns, fill_value=0)

# CuPy로 변환 (numpy 배열을 CUDA GPU 메모리로 복사)
stime = time.time()
a_gpu = cp.asarray(data_encoded.to_numpy(dtype='float32'))  # cupy 배열 객체 
b_gpu = cp.asarray(synth_encoded.to_numpy(dtype='float32')) # cupy 배열 객체
# 중복 체크
# cnt = 0
# for i in range(b_gpu.shape[0]):
#     ## 각 원본 행이 모든 컬럼에서 일치 하는지 -> 원본 중 하나라도 완전 일치 하는 행 있는지
#     if cp.any(cp.all(b_gpu[i] == a_gpu, axis=1)):  
#         cnt += 1


# cp.isin을 사용하여 일치 여부 계산
matching_rows = cp.isin(a_gpu, b_gpu)
list = cp.where(cp.isin(a_gpu, b_gpu) == False)[0]
for x in list:
    print(x)

# False인 값만 추출 (a_gpu에서 b_gpu에 포함되지 않는 값)
non_matching_rows = a_gpu[~matching_rows]

# 중복률 계산 (중복된 값의 비율)
unique_non_matching_rows = cp.unique(non_matching_rows)  # 중복 제거된 값들
non_matching_count = non_matching_rows.shape[0]  # False인 값들의 총 개수
print(non_matching_count)
unique_non_matching_count = unique_non_matching_rows.shape[0]  # 중복을 제거한 값들의 개수
print(unique_non_matching_count)
print(((1-0.5384615384615384) - 0.47049670365668894)*48842)
# 중복률 계산
duplicate_ratio = 1 - (unique_non_matching_count / non_matching_count)
etime    = time.time()

print("CuPy 방식 시간:", etime - stime)
print("중복률:",duplicate_ratio)
# CuPy 방식 시간: 15.944603443145752
# 중복률: 0.47049670365668894

6959
10659
17609
19353
20417
24791
28474
40726
40819
41812
42117
48034
48473
13
6
-437.53846153846155
CuPy 방식 시간: 0.027443408966064453
중복률: 0.5384615384615384


In [48]:
import pandas as pd
import cupy as cp
import numpy as np
import time
import hashlib

# 1. 원핫 인코딩
data_encoded = pd.get_dummies(data)
synth_encoded = pd.get_dummies(synth_data)

# 2. 컬럼 통일 및 순서 보장
all_columns = sorted(set(data_encoded.columns).union(set(synth_encoded.columns)))
data_encoded = data_encoded.reindex(columns=all_columns, fill_value=0).astype(np.uint8)
synth_encoded = synth_encoded.reindex(columns=all_columns, fill_value=0).astype(np.uint8)
# 3. 행 단위 해시 함수
def hash_row(row):
    return np.uint64(int(hashlib.md5(row.tobytes()).hexdigest(), 16) & 0xFFFFFFFFFFFFFFFF)

# 4. 해시값으로 변환
data_hashes = np.array([hash_row(row) for row in data_encoded.to_numpy()])
synth_hashes = np.array([hash_row(row) for row in synth_encoded.to_numpy()])
stime = time.time()

# 6. CuPy 비교
a_gpu = cp.asarray(data_hashes)
b_gpu = cp.asarray(synth_hashes)
matched_gpu = cp.isin(b_gpu, a_gpu)
etime = time.time()
# 7. 결과
count = int(cp.sum(matched_gpu))
duplication_ratio = count / len(b_gpu)
print(" CUPY :" ,etime - stime)
print("중복된 고유 행 수:", count)
print("중복률:", duplication_ratio)



 CUPY : 0.016727447509765625
중복된 고유 행 수: 20137
중복률: 0.4122886040702674


- numpy : 안에서 C, 포트란?? 으로 돌아서.빠름
- list
- 1) 문제 해결 시간 단축 노력 해보기~
- 2) cython? 은 어려움

In [56]:
# 1. 범위 계산 함수
def compute_ranges(original_df, dtypes):
    ranges = []
    for col in original_df.columns:
        if dtypes[col] == 'category':
            ranges.append(0)
        else:
            col_data = original_df[col].astype(float)
            ranges.append(col_data.max() - col_data.min())
    return dict(zip(original_df.columns, ranges))

# 2. 원핫 range 계산
def get_onehot_ranges(data_encoded, original_columns, raw_ranges):
    onehot_ranges = []
    for col in data_encoded.columns:
        base_col = col.split('_')[0]
        range_val = raw_ranges.get(base_col, 1)
        onehot_ranges.append(0 if range_val == 0 else 1)
    return np.array(onehot_ranges, dtype=np.float32)

# 3. CuPy 기반 originality 계산
def calculate_originality_score_cupy(origin_np, synth_np, ranges_np):
    origin = cp.asarray(origin_np)
    synth = cp.asarray(synth_np)
    ranges = cp.asarray(ranges_np)
    
    n_s, k = synth.shape
    n_o = origin.shape[0]
    cnt_satisfied = 0
    stime = time.time()

    for i in range(n_s):
        y = synth[i]

        diff = cp.abs(origin - y)
        normed = cp.where(ranges == 0, (origin != y).astype(cp.float32), diff / ranges)
        dists = cp.mean(normed, axis=1)
        d_s = cp.min(dists)
        jj = cp.argmin(dists)

        mask = cp.arange(n_o) != jj
        origin_others = origin[mask]
        ref = origin[jj]

        diff_o = cp.abs(origin_others - ref)
        normed_o = cp.where(ranges == 0, (origin_others != ref).astype(cp.float32), diff_o / ranges)
        dists_o = cp.mean(normed_o, axis=1)
        d_o = cp.min(dists_o)

        if d_s < d_o:
            cnt_satisfied += 1

    etime = time.time()
    score = cnt_satisfied / n_s

    print(f"\n✅ Originality Score (d_s < d_o): {score:.4f}")
    print(f"✔️ {cnt_satisfied} / {n_s} 합성 샘플이 조건 만족")
    print(f"⏱️ 실행 시간: {etime - stime:.2f}초")
    return score


In [57]:
# 4. ranges 계산 (원본 기준)
raw_ranges = compute_ranges(data, dtypes)

# 5. 원핫 인코딩
data_encoded = pd.get_dummies(data)
synth_encoded = pd.get_dummies(synth_data)

# 6. 컬럼 통일
all_columns = sorted(set(data_encoded.columns).union(set(synth_encoded.columns)))
data_encoded = data_encoded.reindex(columns=all_columns, fill_value=0).astype(np.float32)
synth_encoded = synth_encoded.reindex(columns=all_columns, fill_value=0).astype(np.float32)

# 7. onehot에 대한 range 매핑
ranges_np = get_onehot_ranges(data_encoded, data.columns, raw_ranges)

# 8. NumPy 변환
origin_np = data_encoded.to_numpy()
synth_np = synth_encoded.to_numpy()

# 9. originality score 계산
originality_score = calculate_originality_score_cupy(origin_np, synth_np, ranges_np)

if originality_score >= 0.5:
    print("🎯 Originality 기준 만족")
else:
    print("⚠️ Originality 기준 미달")




✅ Originality Score (d_s < d_o): 0.4479
✔️ 21877 / 48842 합성 샘플이 조건 만족
⏱️ 실행 시간: 112.32초
⚠️ Originality 기준 미달
